# YggDrasil — One Notebook for Everyone

This notebook covers **three levels**: **User** (just run), **Developer** (inspect and modify pipelines), **Enthusiast** (build your own nodes and graphs).

**Setup:** Run the next cell once. You need: `pip install -e .` and `pip install torch torchvision diffusers transformers`.

In [ ]:
import sys
from pathlib import Path
# Find repo root (directory that contains 'yggdrasil' package)
REPO_ROOT = Path.cwd()
while REPO_ROOT != REPO_ROOT.parent and not (REPO_ROOT / 'yggdrasil').is_dir():
    REPO_ROOT = REPO_ROOT.parent
if str(REPO_ROOT) not in sys.path:
    sys.path.insert(0, str(REPO_ROOT))
print('Repo root:', REPO_ROOT)

---
## Part 1: For the User

Generate images in **three ways**: UI, one script, or a few lines of code.

### 1.1 Three commands (no code)

In a terminal:
- `pip install -e .`  (from repo root)
- `yggdrasil ui`  → open the Gradio UI in the browser
- Or: `python examples/images/sd15/run_txt2img.py`  → saves an image to `out/`

### 1.2 High-level API (InferencePipeline)

Like Diffusers: load a pipeline and call it.

In [ ]:
from yggdrasil.pipeline import InferencePipeline

pipe = InferencePipeline.from_template('sd15_txt2img', device='cuda')
output = pipe('a beautiful cat on a windowsill', num_steps=28, seed=42)
img = output.images[0]
Path('out').mkdir(exist_ok=True)
img.save('out/user_demo.png')
print('Saved out/user_demo.png')

In [ ]:
# From Hugging Face model ID
# pipe = InferencePipeline.from_pretrained('runwayml/stable-diffusion-v1-5', device='cuda')
# output = pipe('a cat', num_steps=28, seed=42)

# List available templates
for name, info in InferencePipeline.list_available().items():
    print(name, '—', info.get('description', '')[:60])

---
## Part 2: For the Developer

Work with the **graph**: inspect nodes and edges, modify, export.

In [ ]:
from yggdrasil.core.graph.graph import ComputeGraph

graph = ComputeGraph.from_template('sd15_txt2img', device='cuda')
print('Name:', graph.name)
print('Nodes:', list(graph.nodes.keys()))
print('Inputs:', list(graph.graph_inputs.keys()))
print('Outputs:', list(graph.graph_outputs.keys()))
print('Metadata:', graph.metadata)

In [ ]:
# Edges: who connects to whom
for e in graph.edges:
    print(f'  {e.src_node}.{e.src_port} -> {e.dst_node}.{e.dst_port}')

In [ ]:
# Execute (high-level: auto noise, timesteps)
outputs = graph.execute(prompt='a developer cat', num_steps=20, seed=123)
print('Output keys:', list(outputs.keys()))

In [ ]:
# Nested structure: denoise_loop contains the step graph
loop = graph.nodes.get('denoise_loop')
if loop and hasattr(loop, 'graph'):
    inner = loop.graph
    print('Inner step graph nodes:', list(inner.nodes.keys()))

In [ ]:
# Mermaid diagram
print(graph.visualize())

---
## Part 3: For the Enthusiast

Create **your own block** and build a **graph from scratch**.

### 3.1 Define a custom block

Every block: `declare_io()` + `process()`, and `@register_block`.

In [ ]:
from yggdrasil.core.block.base import AbstractBaseBlock
from yggdrasil.core.block.port import InputPort, OutputPort
from yggdrasil.core.block.registry import register_block

@register_block('demo/identity')
class MyIdentityBlock(AbstractBaseBlock):
    block_type = 'demo/identity'

    @classmethod
    def declare_io(cls):
        return {
            'x': InputPort('x'),
            'y': OutputPort('y'),
        }

    def process(self, **kwargs):
        return {'y': kwargs['x']}

print('Registered: demo/identity')

### 3.2 Build a tiny graph from scratch

Add nodes, connect ports, expose inputs/outputs.

In [ ]:
from yggdrasil.core.graph.graph import ComputeGraph
from yggdrasil.core.graph.executor import GraphExecutor

g = ComputeGraph('demo')
g.add_node('identity', MyIdentityBlock({'type': 'demo/identity'}))
g.expose_input('in', 'identity', 'x')
g.expose_output('out', 'identity', 'y')

result = GraphExecutor().execute(g, in=42)
print('Result:', result)

### 3.3 Use BlockBuilder to load registered blocks

Templates use `BlockBuilder.build({'type': 'conditioner/clip_text', ...})`. You can do the same for any registered type.

In [ ]:
from yggdrasil.core.block.builder import BlockBuilder

# Build from config (same as templates do)
block = BlockBuilder.build({'type': 'demo/identity'})
print('Built:', block.block_type)

---
## Summary

| Level       | What you use | Use case |
|------------|--------------|----------|
| **User**   | `yggdrasil ui` or `run_txt2img.py` or `InferencePipeline.from_template()` + `pipe(...)` | Generate images, no graph knowledge |
| **Developer** | `ComputeGraph.from_template()`, `.nodes`, `.edges`, `.execute()`, `.visualize()`, `.to_yaml()` | Inspect, tweak, export pipelines |
| **Enthusiast** | `AbstractBaseBlock`, `@register_block`, `add_node`, `connect`, `expose_input/output` | Custom blocks, new pipelines, research |